In [1]:
##run this first to install package, then delete
conda install webdriver-manager selenium pandas

SyntaxError: invalid syntax (1152895909.py, line 2)

In [3]:
# import libraries
import time 
import json
import random
import requests
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

Run this block below as is

In [4]:
# start by defining the options 
options = webdriver.ChromeOptions() 
options.headless = True # it's more scalable to work in headless mode 
# normally, selenium waits for all resources to download 
# we don't need it as the page also populated with the running javascript code. 
options.page_load_strategy = 'none' 
# this returns the path web driver downloaded 
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
# pass the defined options and service objects to initialize the web driver 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/210964506.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True # it's more scalable to work in headless mode


This stores a list of listing urls based on a desired quantity (each listing page contains 120 listings). I'm keeping the number low for now, just for efficiency but we'll have to discuss how many individual listings we want to try and scrape. 

In [5]:
url_list = []
page = 0
num_pages = 77

# get urls
for x in range(num_pages):
    page += 1 
    url_list.append("https://losangeles.craigslist.org/search/apa?query=los%20angeles#search=1~list~{}~0".format(page))

For each page of listings, we need to grab the urls for each of the 120 individual listings.

In [6]:
listings = []

for url in url_list:
    # create WebElement of Listings Page
    driver.get(url)
    
    driver.implicitly_wait(5)
    
    # find the section the listings are located in
    content = driver.find_element(By.CSS_SELECTOR, "div[class*='cl-results-page'")
    
    # find the section where the link to the property page is located for each 120 listings
    for element in content.find_elements(By.TAG_NAME, "li")[1:]:
        try:
            listings.append(element.find_elements(By.CSS_SELECTOR, "div[class*='result-node-wide']>a")[0].get_attribute("href"))
            
        except (StaleElementReferenceException, NoSuchElementException) as e:
            listings.append('NA')
    
            
    # add delay (not sure about actually length of time delays necessary)
    time.sleep(random.randint(1, 3))
    
listingsDf = pd.DataFrame(listings)
listingsDf.rename(columns = {0:"urls"}, inplace = True)
# limiting size for testing
listingsDf = listingsDf
# check DF
listingsDf.head()

,urls
0,https://losangeles.craigslist.org/lac/apa/d/lo...
1,https://losangeles.craigslist.org/lac/apa/d/lo...
2,https://losangeles.craigslist.org/lac/apa/d/lo...
3,https://losangeles.craigslist.org/lac/apa/d/lo...
4,https://losangeles.craigslist.org/lac/apa/d/lo...


Now we will scrape each webpage from the list of urls. This is where most of our development and discussion needs to occur. I've placed a temporary simple test in there for now. 

In [7]:
#listingsDf[listingsDf.urls == 'NA']

In [8]:
listingsDf_trim = listingsDf[:1000]

In [9]:
description = []
latitude = []
longitude = []
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)


for url in listingsDf_trim['urls']:
    # create WebElement of listing URL
    driver.get(url)
    
    driver.maximize_window() # For maximizing window
    driver.implicitly_wait(5) # gives an implicit wait for 20 seconds
   
    # grab description of listing and add to list
    try:
        #description
        description.append(WebDriverWait(driver, 5,ignored_exceptions=ignored_exceptions)\
                        .until(expected_conditions.presence_of_element_located((By.ID, "postingbody"))).text)
        
        #lat
        latitude_element = WebDriverWait(driver, 5, ignored_exceptions=ignored_exceptions).until(
                        expected_conditions.presence_of_element_located((By.ID, "map"))
                        )
        latitude_value = latitude_element.get_attribute("data-latitude")
        latitude.append(latitude_value)
                
        #long
        longitude_element = WebDriverWait(driver, 5, ignored_exceptions=ignored_exceptions).until(
                        expected_conditions.presence_of_element_located((By.ID, "map"))
                        )
        longitude_value = longitude_element.get_attribute("data-longitude")
        longitude.append(longitude_value)
        
    except (StaleElementReferenceException, NoSuchElementException, TimeoutException) as e:
        description.append('NA')
        latitude.append('NA')
        longitude.append('NA')
    
    time.sleep(random.randint(1, 3))
    
listingsDf_trim['description'] = description
listingsDf_trim['lat'] = latitude
listingsDf_trim['long'] = longitude

/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/387396221.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listingsDf_trim['description'] = description
/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/387396221.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listingsDf_trim['lat'] = latitude
/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/387396221.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [10]:
listingsDf_trim

,urls,description,lat,long
0,https://losangeles.craigslist.org/lac/apa/d/lo...,QR Code Link to This Post\nshow contact info\n...,34.102708,-118.344810
1,https://losangeles.craigslist.org/lac/apa/d/lo...,Medici is the premier resort apartment communi...,34.055900,-118.266600
2,https://losangeles.craigslist.org/lac/apa/d/lo...,show contact info\n1545 N. Bronson Ave.\nHolly...,34.099350,-118.318213
3,https://losangeles.craigslist.org/lac/apa/d/lo...,QR Code Link to This Post\nshow contact info F...,34.067106,-118.312909
4,https://losangeles.craigslist.org/lac/apa/d/lo...,show contact info\n1545 N. Bronson Ave.\nHolly...,34.099350,-118.318213
...,...,...,...,...
995,https://losangeles.craigslist.org/lac/apa/d/lo...,Welcome Home to Canyon Drive Manor! Our beauti...,34.102967,-118.316377
996,https://losangeles.craigslist.org/wst/apa/d/lo...,"Studio 1 Bathroom apartment.\n\nSpacious, brig...",34.057982,-118.421214
997,https://losangeles.craigslist.org/wst/apa/d/lo...,"Spacious And Bright 2 Bedroom, 2 Bath Dual Mas...",34.057982,-118.421214
998,https://losangeles.craigslist.org/wst/apa/d/lo...,"Westwood 2 Bedroom, 2 Bath Apartment Close To ...",34.062372,-118.441479


In [104]:
listingsDf_trim_1000 = listingsDf_trim

In [105]:
listingsDf_trim_100 = listingsDf[:100]

In [106]:
import re

def remove_and_clean(text):
    text = re.sub(("\n"), " ", text)
    text = text.lower()
    return text

listingsDf_trim['description'] = listingsDf_trim['description'].apply(remove_and_clean)

/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/568058012.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listingsDf_trim['description'] = listingsDf_trim['description'].apply(remove_and_clean)


In [107]:
listingsDf_trim.drop_duplicates(subset='description', inplace=True)

/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/2941071184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listingsDf_trim.drop_duplicates(subset='description', inplace=True)


In [108]:
import geopandas as gpd

parks = gpd.read_file('county_parks.geojson')
parks

OBJECTID   UNIT_ID     LMS_ID  \
0            1      30.0    23603.0   
1            2      56.0        NaN   
2            3      62.0  3000015.0   
3            4      72.0    22715.0   
4            5      78.0    22860.0   
...        ...       ...        ...   
3007      3009    6970.0  3000052.0   
3008      3010    6229.0  3000074.0   
3009      3011  100023.0  3000071.0   
3010      3012    7277.0  3000064.0   
3011      3013       NaN        NaN   

                                        PARK_NAME  \
0                                  El Barrio Park   
1                                  El Dorado Park   
2                            Atlantic Avenue Park   
3                               El Escorpion Park   
4            El Marino Park/ El Marino Elementary   
...                                           ...   
3007  Eaton Canyon Natural Area and Nature Center   
3008       Earvin "Magic" Johnson Recreation Area   
3009                               Hollywood Bowl   
3010                William S. Hart Regional Park   
3011     Greater Whittier Regional Aquatic Center   

                                      PARK_LBL         ACCESS_TYP   RPT_ACRES  \
0                               El Barrio Park        Open Access    2.093000   
1                               El Dorado Park        Open Access    5.144000   
2                         Atlantic Avenue Park        Open Access    1.960000   
3                            El Escorpion Park        Open Access   66.914000   
4     El Marino Park Elementary/El Marino Park        Open Access    3.200000   
...                                        ...                ...         ...   
3007         Eaton Canyon Park and Nature Ctr.        Open Access  219.087769   
3008          Earvin 'Magic' Johnson Rec. Area        Open Access  120.660000   
3009                            Hollywood Bowl        Open Access   77.430000   
3010             William S. Hart Regional Park        Open Access  155.869048   
3011  Greater Whittier Regional Aquatic Center  Restricted Access    2.347900   

       GIS_ACRES              AGNCY_NAME AGNCY_LEV  ...  CENTER_LON  \
0       2.092992      Claremont, City of      City  ... -117.704423   
1       5.143217      Lancaster, City of      City  ... -118.122109   
2       1.959610  Los Angeles, County of    County  ... -118.155201   
3      66.338029    Los Angeles, City of      City  ... -118.664169   
4       1.650439    Culver City, City of      City  ... -118.399233   
...          ...                     ...       ...  ...         ...   
3007  219.087769  Los Angeles, County of    County  ... -118.096038   
3008  120.660543  Los Angeles, County of    County  ... -118.260350   
3009   77.428979  Los Angeles, County of    County  ... -118.339429   
3010  155.869048  Santa Clarita, City of      City  ... -118.526456   
3011    2.347900  Los Angeles, County of    County  ... -118.078681   

                      ADDRESS         CITY    ZIP  \
0          400 Claremont Blvd    Claremont  91711   
1              44501 5th St E    Lancaster  93535   
2       570 S. Atlantic Blvd.  Los Angeles  90022   
3       6801  Sunset Ridge Ct   West Hills  91307   
4               11450 Port Rd  Culver City  90230   
...                       ...          ...    ...   
3007     1750 N. Altadena Dr.     Pasadena  91001   
3008  905 E. El Segundo Blvd.  Los Angeles  90059   
3009    2301 N. Highland Ave.  Los Angeles  90068   
3010    24151 N. Newhall Ave.      Newhall  91321   
3011       8028 Pioneer Blvd.     Whittier  90606   

                                                                                                                                                                                                                                                                                                                                                                                                                                                     

There are also a bunch of neighborhoods (and transit stations) in Los Angeles that are named after a park.. tricky. We wanted to take these listings out and any terms luxury developers use to reffer to their ammenities that aren't public (for example "rooftop park") this process of identifying terms required a lot of trial and errror to read through sample posts and identify trends

In [159]:
excluded = ['highland park',
           'hancock park',
           'echo park and silverlake',
            'silverlake and echo park',
            'silverlake, echo park',
            'echo park, silverlake',
           'rancho park',
           'macarthur park station',
           'south park',
           'rooftop park',
           'hollywood bowl']

There are some listings that allude to a park or many parks without naming the exact one. We wanted ot inlcude those listings in our analysis so we created a list of terms that allude to parks called "park_terms". This is tricky becuase you want to avoid counting anything that is referring to parking. 

In [171]:
park_terms = ['hike', 
                'hiking',
                'trail', 
                'trails', 
                'community park',
                'community parks', 
                'local park',
                'local parks',
                 'parks nearby',
                'public parks',
                'public park',
                'echo park lake',
                'park access',
                'parks',
              'recreational parks',
              'a park nearby',
              'the park',
              'park nearby',
              'griffith park',
              'exposition park',
              'recreation center',
              'rergional park',
              'griffith park,'
                 ]

Now we want to read through all the descriptions and identify those that mention a park. We will do this by assigning listings that mention a park with a "true" vlaue in a new column Park_TF. This function reads descriptions and makes every listing that has a park name from the parks data file "true" unless its one of the excluded terms for neigborhoods named after parks ie highland park, then adds all the posts that have one of the park terms as true.

In [172]:
listingsDf_trim['park_TF'] = listingsDf_trim['description'].apply(
    lambda x: any(substring in x for substring in parks['PARK_NAME'].str.lower()) and not any(substring in x for substring in excluded)or any(substring in x for substring in park_terms))

/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/1419522507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listingsDf_trim['park_TF'] = listingsDf_trim['description'].apply(


We also want to know why the function named the listing true so we added another column called park_T_why which fills in the terms that flagged the description as mentioning a park

In [173]:
listingsDf_trim['park_T_why'] = listingsDf_trim['description'].apply(
    lambda x: [substring for substring in parks['PARK_NAME'].str.lower() if substring in x and substring not in excluded] or [substring for substring in park_terms if substring in x]
)

/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_77039/3464894427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listingsDf_trim['park_T_why'] = listingsDf_trim['description'].apply(


<div id="map" class="viewposting leaflet-container leaflet-retina leaflet-fade-anim leaflet-grab leaflet-touch-drag" data-latitude="33.959712" data-longitude="-118.419410" data-accuracy="10" tabindex="0">

lets see what we are capturing. first count the numebr of listing with parks mentioend using value counts

In [174]:
listingsDf_trim.park_TF.value_counts()

False    783
True     151
Name: park_TF, dtype: int64

now expand view and read through a sample of the descriptions with park mentioned. if enerything looks good thats aweosme, if not you might have to ammend your list of excluded and included terms

In [175]:
pd.set_option('max_colwidth', None)
listsingsDf_trim_parks = listingsDf_trim[listingsDf_trim['park_TF'] == True]

In [176]:
listsingsDf_trim_parks.sample(15)

,urls,description,lat,long,park_TF,FoundSubstrings,park_TF_why,park_T_why
138,https://losangeles.craigslist.org/lac/apa/d/los-angeles-327-ave-nd-floor-hardwood/7620491504.html,"2 bedroom 1 bath, 2nd floor apartment $2400.00 rent, $2400.00 security deposit, $30.00 credit check per applicant pet friendly! $25/month pet rent plus $500 pet deposit upon owner approval address: 327 s ave 60 #211 los angeles, ca 90042 apartment features • fresh paint • stove included • vinyl in kitchen & bathroom • hardwood laminate in living room • new carpet in bedrooms • built in shelves in living room • air conditioning wall unit • abundant natural light complex features • on-site laundry • on-site manager • gated entry • gated pool lease terms • 12 month lease • no smoking • utilities include: water contact information viewing or leasing available 7 days a week to view during the daytime or weekend call ginda show contact info professionally managed by gentry real estate investments, llc 5330 n. figueroa st #101 los angeles, ca 90042 show contact info ""fair housing and equal opportunity"" located in highland park nearby cities: eagle rock - south pasadena montecito heights - mt washington - cypress park northeast los angeles - downtown la thank you for looking!",34.108816,-118.186884,True,"[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]",[park nearby],[park nearby]
301,https://losangeles.craigslist.org/lac/apa/d/los-angeles-studio-in-koreatown-exposed/7629630046.html,"square footage is approximate. price and availability may change. ancelle apartments 701 south gramercy drive los angeles, ca 90005 call: show contact info the ancelle brings the romance of the 20's to your everyday life. step onto the grounds and find yourself in a park like setting with a stately three-tier fountain centerpiece. on the way to your renovated apartment, you'll pass through the lofty foyer, complete with baby grand piano. high ceilings, exposed brick and original hardwood floors create a luxurious backdrop for your home. storage will not be a problem, these units include built in vanities and lots of closet space. apartment features - loft* - exposed brick - dishwasher* - stainless steel appliances* - microwave* - hardwood floors - 9 foot ceilings* - den/study* - ceiling fans* - refrigerator - balcony/patio* - mirrored closet doors* - tile floors in kitchen/bathroom - quartz countertops - concrete floors* - extra storage - air conditioning - walk-in closets - central air/heating* * with select apartment homes community features - renovated lobby - cable available - assigned parking* - easy access to freeways - on-site maintenance - gated access - courtyard fountain - elevator - airport transportation - public parks nearby - laundry facility - metro stops - gazebo - beautiful landscaping - access to public transportation - easy access to shopping contact information viewing or leasing information call: show contact info location benefits ( near 10 & 101 freeway ) nearby cities: greater wilshire/hancock park, mid-wilshire, mid city, wilshire center, westlake, pico union, koreatown, miracle mile",34.059918,-118.313286,True,"[False, False, False, False, False, False, False, False, True, True, True, False, False, False, True, False, True, False, False]","[parks nearby, public parks, public park, parks, a park]","[parks nearby, public parks, public park, parks]"
624,https://losangeles.craigslist.org/lac/apa/d/los-angeles-newly-renovated-studio-in/7628608983.html,"newly renovated with a bright modern kitchen with new countertops, lots of storage, all new appliances. hardwood floors, new finishes and lighting throughout, nice big closets, and lovely bathroom. all units have all new sound reducing windows and a view of the street, so lots of natural light. only a block to the metro and convenient to usc, exposition park museums and the staples center. a quick hop on the 10 makes getting to 

In [177]:
listingsDf_trim = listingsDf_trim[listingsDf_trim['lat'] != 'NA']
test = gpd.GeoDataFrame(listingsDf_trim, geometry=gpd.points_from_xy(listingsDf_trim.long, listingsDf_trim.lat), crs="EPSG:4326")


In [1]:
# import libraries
import plotly.express as px
import matplotlib.pyplot as plt


# create scatter map
fig = px.scatter_mapbox(test, lat=test.geometry.y, lon=test.geometry.x, color="park_TF",
                               mapbox_style="carto-positron",
                               #range_color=range_color,
                               zoom=9,
                               center = {"lat": 34, "lon": -118.4},
                               opacity=.8,

                               )

# options on the layout
fig.update_layout(
        width = 900,
        height = 700,
        title = "Listings",
        title_x = .5
    )
fig.show()

NameError: name 'test' is not defined

In [19]:
listingsDf_trim.to_pickle("./dummy.pkl")